<a href="https://colab.research.google.com/github/k984530/-url/blob/main/%EC%95%BD%EA%B3%BC%EC%82%AC%EC%A0%84.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**환경설정**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install torch torchvision
!pip install numpy
!pip install opencv-python
!pip install imgaug
!pip install PIL
!pip install tqdm
!pip install codecs
!pip install json
!pip install matplotlib

In [ ]:
# !pip install PIL error 해결 방안
!pip install Pillow

In [ ]:
import torch
import torchvision.models as models

# 학습한 파라미터가 저장된 파일 경로
trained_params_path = '/content/drive/MyDrive/data/pill_resnet152_dataclass01_aug0.pt'

# 사전 학습된 ResNet-152 모델 아키텍처 가져오기
resnet152 = models.resnet152(pretrained=True)

# 학습한 파라미터 로드
trained_params = torch.load(trained_params_path)
resnet152.load_state_dict(trained_params)

# 이미지에 모델 적용 및 예측 수행
from torchvision import transforms
from PIL import Image

# 이미지 전처리를 위한 변환 정의
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# 이미지 불러오기 및 전처리
image_path = 'path_to_your_image.jpg'
image = Image.open(image_path)
input_tensor = preprocess(image)
input_batch = input_tensor.unsqueeze(0)  # 모델에 들어갈 배치 차원 추가

# 모델에 이미지 전달하여 예측 수행
resnet152.eval()  # 모델을 평가 모드로 설정
with torch.no_grad():
    output = resnet152(input_batch)

# 예측 결과 출력
print(output)


## **copy_crop_pill_from_org.py**

In [ ]:
import cv2
from pathlib import Path
import shutil
import numpy as np


list_dir_pill_org = [
    # r'G:\경구약제 DB\0.단일 경구약제 1000종\경구약제 1000종 추가 DB',
    # r'G:\경구약제 DB\0.단일 경구약제 1000종\경구약제 1000종 추가 DB_1',
    r'G:\경구약제 DB\0.단일 경구약제 1000종\경구약제 1000종 추가 DB_2',
]


# JSON 파일과 이미지 파일을 읽어서, 이미지를 자르고, 새로운 크기로 resize 후, 새로운 경로에 저장
def copy_crop_pill_from_json(args, pathfile_src_json, pathfile_src_png, pathfile_dest_json, pathfile_dest_png):

    # 이미지 파일이 없으면 함수를 종료
    if not pathfile_src_png.exists() :
        args.logger.info(f"file does'nt exist : {str(pathfile_src_png)}")
        return

    # JSON 파일을 새로운 경로에 복사
    shutil.copyfile(str(pathfile_src_json), str(pathfile_dest_json))

    # crop
    dict_pill_info = utils.read_dict_from_json(str(pathfile_src_json))
    bbox = dict_pill_info['annotations'][0]['bbox']
    np_bbox = np.array([[bbox[0], bbox[1]], [bbox[0] + bbox[2], bbox[1] + bbox[3]]])
    np_center = np.average(np_bbox, axis=0)
    diff =  np.max(np_bbox[1] - np_bbox[0])
    np_low = np_center - diff/2
    np_high = np_center + diff/2
    np_high = np_high.astype(np.int32)
    np_low = np_low.astype(np.int32)

    image_cv = utils.open_opencv_file(str(pathfile_src_png))
    image_cv_cropped = image_cv[np_low[1]:np_high[1], np_low[0]:np_high[0]]
    image_cv_resized = cv2.resize(image_cv_cropped, (args.size_image, args.size_image))

    utils.save_opencv_file(image_cv_resized, str(pathfile_dest_png))

# 경로 list_dir_pill_org 에 있는 모든 약제 이미지를 처리
def copy_crop_pill_from_org(args):
    '''
    list_dir_pill_org 의 directory은  알약  dir들을 여러 개 포함하고,
    알약 dir에는 조건에 따라, 1200여개의 png가 있다.
    이를  아래 pathdir_dest dir에 모든다.
    단 이미지를 bbox기준으로, 224x224으로 crop한다.
    :param args:
    :return:
    '''
    # 새로운 이미지를 저장할 경로
    pathdir_dest = Path(r'G:\proj_pill_data\pill_data_croped')

    for dir_pill_org in list_dir_pill_org :
        list_count = [ 1 for _ in Path(dir_pill_org).iterdir()]
        count_dir = len(list_count)
        index_dir = 0
        for pathdir_pill_class_json in Path(dir_pill_org).iterdir():
            print(f'working dir index is { index_dir}/{count_dir}, {str(pathdir_pill_class_json)}')
            index_dir += 1
            if not 'json' in pathdir_pill_class_json.name or not pathdir_pill_class_json.is_dir():
                continue

            no_json = pathdir_pill_class_json.name.split('_')[0]
            pathdir_pill_class = pathdir_pill_class_json.with_name(no_json)
            pathdir_dest_class = pathdir_dest.joinpath(no_json)
            pathdir_dest_class.mkdir(exist_ok=True)

            for i,  pathfile_pill_json in enumerate(pathdir_pill_class_json.glob('*.json')):
                # if i == 0 :
                #     shutil.copy(pathfile_pill_json,pathdir_dest )
                pathfile_pill_png = pathdir_pill_class.joinpath( pathfile_pill_json.stem + '.png')
                copy_crop_pill_from_json(args, pathfile_pill_json, pathfile_pill_png, pathdir_dest_class.joinpath(pathfile_pill_json.stem + '.json'), pathdir_dest_class.joinpath(pathfile_pill_json.stem + '.png'))

# if __name__ == '__main__':
#     # job = 'hrnet_w64'
#     job = 'resnet152'
#     args = get_cli_args(job=job, run_phase='train', aug_level=1 )
#     args.logger = utils.create_logging(args.file_log)
#     copy_crop_pill_from_org(args)
#     print('done')

ModuleNotFoundError: ignored

## **gen_pill.py**

In [ ]:
import numpy as np
import cv2
from imgaug import augmenters as iaa
from pathlib import Path
import utils
import make_label_sharpness
from make_pill_class_list import get_pillid_from_pillfile
from PIL import Image

# https://github.com/aleju/imgaug

list_level_aug_geo = [
    [0, iaa.Identity()],
]

list_level_aug_geo_scale = [
    [0, iaa.Identity()],
    [1, iaa.Affine(scale=0.85)],
    [2, iaa.Affine(scale=0.80)],
    [3, iaa.Affine(scale=0.75)],
    [3, iaa.Affine(scale=0.70)],

]

list_level_aug_geo_rotate = [
    [0,iaa.Identity()],
]

list_level_aug_non_geo = [
    [0,iaa.Identity()],
    [1,iaa.GaussianBlur(sigma=1.0)],
    [2,iaa.Dropout(p=0.05)],
    [2,iaa.CoarseDropout(0.02, size_percent=0.5)],
    [2,iaa.SaltAndPepper(0.1)],
    [2,iaa.JpegCompression(compression=10)],
    [3,iaa.ChangeColorTemperature(1000)],
    [3,iaa.ChangeColorTemperature(4000)],
    [3,iaa.Snowflakes(flake_size=0.1, speed=0.01)],
    [3,iaa.GammaContrast(2.0)],
]

def rescaleToFit(np_image, np_point, fit_x, fit_y, scale_limit = 0.8):      #  scale_limit is for image rotation with nppoint.
    np_image = np_image.copy()
    h, w, c = np_image.shape
    # show_cvimage(np_image)
    scale = fit_x / w       # 먼저 x scale 을 정하고,
    if scale * h > fit_y :  # scale이 h 에는  over height.
        scale =  fit_y /  h # y scale으로 정한다.

    scale = scale * scale_limit
    np_image = cv2.resize(np_image, (int(round(w * scale)), int(round(h * scale)) ))
    x_scale = np_image.shape[1] / w
    y_scale = np_image.shape[0] / h

    np_images_back = np.zeros((fit_y, fit_x, c), dtype=np.uint8)
    x_offset = (fit_x - np_image.shape[1]) // 2
    y_offset = (fit_y - np_image.shape[0]) // 2

    np_images_back[y_offset:y_offset + np_image.shape[0], x_offset:x_offset+np_image.shape[1], : ] = np_image


    if len(np_point) > 0:
        np_point *= (x_scale, y_scale)

        np_point[np_point < 0 ] += -1000
        np_point += [x_offset, y_offset ]
        np_point[np_point < 0 ] = -1

    # draw_limbs_on_image(np_images_back,np_point, list_limb_digit0, (0, 0, 255), 2 , False )
    # show_cvimage(np_images_back)

    return np_images_back, np_point


list_aug_geo = []
list_aug_geo_scale=[]
list_aug_geo_rotate = []
list_aug_non_geo = []

class Gen_Digit():
    def __init__(self,args, dir_dataset,run_phase):
        print(f'dataset dir is {dir_dataset}')

        self.build_list_gen_based_on_level(args)
        self.args = self.gen_pill_ready(args, dir_dataset, run_phase)

    def build_list_gen_based_on_level(self, args):
        global list_aug_geo, list_aug_geo_scale, list_aug_geo_rotate, list_aug_non_geo

        print(f'run_phase is {args.run_phase}, aug_level is {args.aug_level}')

        for i, (level, aug) in enumerate( list_level_aug_geo):
            aug.name = f'g{i}'
        for i, (level, aug) in enumerate( list_level_aug_geo_scale):
            aug.name = f's{i}'
        for i, (level, aug) in enumerate( list_level_aug_geo_rotate):
            aug.name = f'r{i}'
        for i, (level, aug) in enumerate( list_level_aug_non_geo):
            aug.name = f'n{i}'

        if args.run_phase == 'train' :
            list_aug_geo = [ aug for level, aug in list_level_aug_geo if level <= args.aug_level]
            list_aug_geo_scale = [aug for level, aug in list_level_aug_geo_scale if level <= args.aug_level]
            list_aug_geo_rotate = [aug for level, aug in list_level_aug_geo_rotate if level <= args.aug_level]
            list_aug_non_geo = [aug for level, aug in list_level_aug_non_geo if level <= args.aug_level]
        else:
            list_aug_geo = [iaa.Identity()]
            list_aug_geo_scale = [iaa.Identity()]
            list_aug_geo_rotate = [iaa.Identity()]
            list_aug_non_geo = [iaa.Identity()]

    def gen_pill_ready(self, args, dir_dataset, run_phase):
        global list_aug_geo, list_aug_geo_scale, list_aug_geo_rotate, list_aug_non_geo
        print(f'gen_type is {args.gen_type}, loading data ...')

        path_dir_json = Path(dir_dataset)
        if path_dir_json.is_dir() :
            print(f'Gen :reading directory was not implemented')
            self.len_total = 0

        elif path_dir_json.is_file() and path_dir_json.suffix == '.json' and args.gen_type == 'read_only_image':

            dict_temp = utils.read_dict_from_json(str(path_dir_json))
            self.list_label_path = []
            if args.gen_dataclass_sel in [ 'dataclass0', 'dataclass01'] :
                if run_phase == 'train' :
                    self.list_label_path += dict_temp.get('pngfile_class0_train', [])
                    print(f'label_path was loaded from   <<< pngfile_class0_train >>>')
                elif run_phase == 'valid' :
                    self.list_label_path += dict_temp.get('pngfile_class0_valid', [])
                    print(f'label_path was loaded from   <<< pngfile_class0_valid >>>')
                else:
                    self.list_label_path += dict_temp.get('pngfile_class0_test', [])
                    print(f'label_path was loaded from   <<< pngfile_class0_test >>>')

            if args.gen_dataclass_sel in [ 'dataclass1', 'dataclass01'] :
                if run_phase == 'train' :
                    self.list_label_path += dict_temp.get('pngfile_class1_train', [])
                    print(f'label_path was loaded from   <<< pngfile_class1_train >>>')
                elif run_phase == 'valid' :
                    self.list_label_path += dict_temp.get('pngfile_class1_valid', [])
                    print(f'label_path was loaded from   <<< pngfile_class1_valid >>>')
                else:
                    self.list_label_path += dict_temp.get('pngfile_class1_test',[])
                    print(f'label_path was loaded from   <<< pngfile_class1_test >>>')

            dict_pillid_label = make_label_sharpness.get_dict_pillid_label(args)
            self.list_label_path = [(dict_pillid_label[get_pillid_from_pillfile(pngfile)], pngfile) for pngfile in self.list_label_path]

            self.list_aug_geo = list_aug_geo
            self.list_aug_geo_scale = list_aug_geo_scale
            self.list_aug_geo_rotate = list_aug_geo_rotate
            self.list_aug_non_geo = list_aug_non_geo

            self.len_list_aug_geo = len(self.list_aug_geo)
            self.len_list_aug_geo_scale = len(self.list_aug_geo_scale)
            self.len_list_aug_geo_rotate = len(self.list_aug_geo_rotate)
            self.len_list_aug_non_geo = len(self.list_aug_non_geo)
            self.len_list_label_path = len(self.list_label_path)

            self.len_total = self.len_list_aug_geo * self.len_list_aug_geo_scale * self.len_list_aug_geo_rotate * self.len_list_aug_non_geo * self.len_list_label_path
            self.div_aug_geo = self.len_list_aug_geo_scale * self.len_list_aug_geo_rotate * self.len_list_aug_non_geo * self.len_list_label_path
            self.div_aug_geo_scale = self.len_list_aug_geo_rotate * self.len_list_aug_non_geo * self.len_list_label_path
            self.div_aug_geo_rotate = self.len_list_aug_non_geo * self.len_list_label_path
            self.div_aug_non_geo = self.len_list_label_path

        print(f"data loading done.  dataset'length is {self.len_total}")
        return args


    def generate_digits_by_index(self,args, index ) :
        if args.gen_type == 'read_only_image' :

            ind = index // self.div_aug_geo
            mod = index % self.div_aug_geo

            aug_geo = self.list_aug_geo[ind]

            ind = mod // self.div_aug_geo_scale
            mod = mod % self.div_aug_geo_scale

            aug_geo_scale = self.list_aug_geo_scale[ind]

            ind = mod // self.div_aug_geo_rotate
            mod = mod % self.div_aug_geo_rotate

            aug_geo_rotate = self.list_aug_geo_rotate[ind]

            ind = mod // self.div_aug_non_geo
            mod = mod % self.div_aug_non_geo

            aug_non_geo = self.list_aug_non_geo[ind]
            label, path_img = self.list_label_path[mod]
            np_image = np.array(Image.open(path_img))

            # np_image= aug_geo(image=np_image)
            # np_image= aug_geo_scale(image=np_image)
            # np_image= aug_geo_rotate(image=np_image)
            # np_image= aug_non_geo(image=np_image)

            iaaaug = iaa.Sequential([aug_geo, aug_geo_scale, aug_geo_rotate, aug_non_geo])
            np_image = iaaaug(image=np_image)

            aug_name = aug_geo.name + aug_geo_scale.name + aug_geo_rotate.name + aug_non_geo.name

            return np_image, label, path_img, aug_name

if __name__ == '__main__' :
    print(f'done')

## **get_cli_args.py**

In [ ]:
import os
import sys
import torch
import argparse
import uuid

is_debug = True
def debugger_is_active() -> bool:
    """Return if the debugger is currently active"""
    gettrace = getattr(sys, 'gettrace', lambda : None)
    return gettrace() is not None

is_debug = debugger_is_active()
# print(f'is_debug is {is_debug}')        # debug 모드로 시작하면,  is_debug == True 이다.

uuid_node = uuid.getnode()

def get_cli_args(job='resnet152', run_phase = 'train', aug_level=0, dataclass='0'):
    #######################################################################################################
    print(f'job={job} run_phase:{run_phase} aug_level:{aug_level}, dataclass:{dataclass} ')
    parser = argparse.ArgumentParser(description=__doc__, formatter_class=argparse.ArgumentDefaultsHelpFormatter, )

    verbose = True
    if os.name == 'nt':
        dir_solution_home = r'D:\proj_pill'
        BATCH_SIZE = 2
        num_workers = 2
        num_threads = 1
        dist_backend = 'gloo'
        if job == 'resnet152':
            BATCH_SIZE = 12

    else:
        dir_solution_home = r'/home/ubuntu/proj/proj_pill'
        num_workers = 4
        num_threads = 2
        dist_backend = 'nccl'
        BATCH_SIZE = 8
        if uuid_node == 274973445269205:        # 광주AI
            BATCH_SIZE = 56                     # train+valid:56 (opt을 올리지 않는다.), train:56
            if job == 'hrnet_w64':
                num_workers = 2
                BATCH_SIZE = 32
            if job == 'resnet152':
                num_workers = 4
                BATCH_SIZE = 64
        elif uuid_node == 274973438730257:      # 광주AI2
            BATCH_SIZE = 768
            if job == 'paf_vgg19':
                BATCH_SIZE = 512


    if run_phase == 'valid'  and run_phase == 'test':
        BATCH_SIZE = 1
        num_workers = 1
        num_threads = 1
        verbose = False

    DIR_DATA = os.path.join(dir_solution_home, 'pill_data')
    DIR_PROJ = os.path.join(dir_solution_home, 'proj_pill')



    # define directory
    dir_pill_class_base = 'pill_data_croped'
    json_pill_label_path_sharp_score = 'pill_label_path_sharp_score.json'
    json_pill_prescription_type = 'pill_prescription_type.json'
    json_pill_class_list = 'pill_class_list.json'

    FITTOSIZE = 224
    gen_type = 'read_only_image'
    dataclass = f'dataclass{dataclass}'

    if job == 'resnet152':
        model_path_in = os.path.join(DIR_PROJ, f'pill_resnet152_{dataclass}_aug{aug_level}.pt')
        model_path = os.path.join(DIR_PROJ, f'pill_resnet152_{dataclass}_aug{aug_level}.pt')



    elif job == 'hrnet_w64':
        model_path_in = os.path.join(DIR_PROJ, f'pill_hrnet_w64_{dataclass}_aug{aug_level}.pt')
        model_path = os.path.join(DIR_PROJ, f'pill_hrnet_w64_{dataclass}_aug{aug_level}.pt')


    #######################################################################################################


    dir_pill_class_base = os.path.join(DIR_DATA, dir_pill_class_base)
    json_pill_label_path_sharp_score = os.path.join(dir_pill_class_base, json_pill_label_path_sharp_score)
    json_pill_prescription_type = os.path.join(dir_pill_class_base, json_pill_prescription_type)
    dir_output = os.path.join(DIR_DATA, 'output')  # output dir for generation from gauge info



    dir_log = './logs'
    file_log = os.path.join(dir_log, f'log-{job}.txt')

    #######################################################################################################
    tqdm_desc_head = f'{job} aug_level:{aug_level} :'
    print(f'BATCH_SIZE:{BATCH_SIZE}, num_workers:{num_workers}, num_threads:{num_threads} ')
    #######################################################################################################
    parser.add_argument('--size_image', default=FITTOSIZE, type=int, help='size of heatmap, vectmap')

    parser.add_argument('--dir_log', default=dir_log, help='tensorboard log directory')
    parser.add_argument('--file_log', default=file_log, help='tensorboard log directory')
    parser.add_argument('--dir_pill_class_base', default=dir_pill_class_base, help='pill class valid directory')
    parser.add_argument('--json_pill_label_path_sharp_score', default=json_pill_label_path_sharp_score, help='pill class sharpness')
    parser.add_argument('--json_pill_prescription_type', default=json_pill_prescription_type, help='pill prescription_type')

    parser.add_argument('--pill_dataset_class0', default=[90, 75], help='dataset camera latitude ')
    parser.add_argument('--pill_dataset_class1', default=[70, 60], help='dataset camera latitude')

    parser.add_argument('--pill_dataset_train_rate', default=0.8, help='dataset train rate')
    parser.add_argument('--pill_dataset_valid_rate', default=0.1, help='dataset valid rate')
    parser.add_argument('--pill_dataset_test_rate' , default=0.1, help='dataset test rate')
    parser.add_argument('--num_classes', default=1000, help='pill dataset class number')

    json_pill_class_list  = os.path.join(dir_pill_class_base, json_pill_class_list )
    parser.add_argument('--json_pill_class_list', type=str, default=json_pill_class_list, help='json file for json_pill_class_list ')

    parser.add_argument('--gen_type', default=gen_type, help='image only or annotation file')
    parser.add_argument('--gen_dataclass_sel', default=dataclass, help='class0, class1, class01')
    parser.add_argument('--cnn_name', default=job, help='classifier name')  # resnet152, hrnet_w64

    # Default settings from https://arxiv.org/abs/1706.02677.
    parser.add_argument('--batches_per_allreduce', type=int, default=1, help='number of batches processed locally before executing allreduce across workers; it multiplies total batch size.')
    parser.add_argument('--fp16_allreduce', action='store_true', default=False, help='use fp16 compression during allreduce')
    parser.add_argument('--use_adasum', action='store_true', default=False, help='use adasum algorithm to do reduction')
    parser.add_argument('--gradient_predivide_factor', type=float, default=1.0, help='apply gradient predivide factor in optimizer (default: 1.0)')
    parser.add_argument('--wd', type=float, default=0.0001, help='weight decay')
    parser.add_argument('--momentum', type=float, default=0.9, help='SGD momentum')
    parser.add_argument('--warmup_epochs', type=float, default=5, help='number of warmup epochs')
    parser.add_argument('--epochs', type=int, default=150, help='number of epochs to train')
    parser.add_argument('--disable_cuda', action='store_true', default=False, help='disables CUDA training')

    parser.add_argument('--base_lr', type=float, default=0.001, help='learning rate for a single GPU')
    parser.add_argument('--lr', '--learning_rate', default=0.001, type=float, metavar='LR', help='initial learning rate')
    parser.add_argument('--pre_lr', type=float, default=1e-4, help='pre learning rate')

    parser.add_argument('--lr_schedule', default=[60, 100, 140], help='lr scheduler')
    # parser.add_argument('--lr_schedule', default=[3, ], help='lr scheduler')
    parser.add_argument('--lr_gamma', default=0.1, help='lr scheduler')
    parser.add_argument('--lr_factor', default=0.1, help='lr scheduler')

    parser.add_argument('--batch_size', default=BATCH_SIZE, type=int, help='batch size')
    parser.add_argument('--allreduce_batch_size', default=8, type=int, help='batch size')
    parser.add_argument('--run_phase',  default=run_phase, help='train , valid')
    parser.add_argument('--optimizer', default='sgd', help='select the optimizer in  sgd, adam, rmsprop')


    parser.add_argument('--model_path', default=model_path, type=str, metavar='DIR', help='path to where the model saved')
    parser.add_argument('--model_path_in', default=model_path_in, type=str, metavar='DIR', help='path to where the model saved')

    #######################################################################################################
    #######################################################################################################
    vgg19_path = os.path.join(DIR_PROJ, 'paf_vgg19_level{aug_level}.pt')

    dir_gauge_json = os.path.join(dir_solution_home, '민성기', 'digitGaugeSamples')
    dir_gauge_json = os.path.join(dir_solution_home, '민성기', 'digitGaugeSamples2')
    dir_gauge_json = os.path.join(dir_solution_home, '민성기', 'digitGaugeSamples3')
    dir_gauge_json = os.path.join(dir_solution_home, '민성기', 'digitGaugeTest')
    dir_class_basic = os.path.join(DIR_DATA, 'digit_class_base')
    dir_class_basic_aug = os.path.join(DIR_DATA, 'digit_class_base_aug')

    dir_digit_back = os.path.join(DIR_DATA, 'backimage')  # back image
    dir_digit_anno = os.path.join(DIR_DATA, 'annotation')  # annotation
    dir_digit_annoimage = os.path.join(DIR_DATA, 'annotationImage')  # annotation + image

    dir_digit_paf = os.path.join(DIR_DATA, 'digit_paf_train')  # annotation + image
    dir_digit_heat = os.path.join(DIR_DATA, 'digit_heat_train')  # annotation + image



    parser.add_argument('--pre_n_images', default=8000, type=int, help='number of images to sampe for pretraining')
    parser.add_argument('--n_images', default=None, type=int, help='number of images to sample')
    parser.add_argument('--duplicate_data', default=None, type=int, help='duplicate data')


    parser.add_argument('--nesterov', dest='nesterov', default=True, type=bool)
    parser.add_argument('--print_freq', default=10, type=int, metavar='N', help='number of iterations to print the training statistics')
    parser.add_argument('--freeze_base', default=0, type=int, help='number of epochs to train with frozen base')
    parser.add_argument('--update_batchnorm_runningstatistics', default=False, action='store_true', help='update batch norm running statistics')
    ########################################################################################################
    # paf parameter

    parser.add_argument('--stride', default=8, type=int, help='stride of heatmap, vectmap')
    parser.add_argument('--train_scale', default=5, type=int, help='stride of heatmap, vectmap')
    parser.add_argument('--num_pts_between_keypoints', default=10, type=int, help='the number of points between two keypoints ')
    parser.add_argument('--ema', default=1e-3, type=float, help='ema decay constant')
    parser.add_argument('--debug_without_plots', default=False, action='store_true', help='enable debug but dont plot')
    parser.add_argument('--category_name', default='digit0', type=str, help='category name inside of annotation to process')
    parser.add_argument('--model_factor', default=8, type=int, help='image downsampler factor')
    parser.add_argument('--heatmap_threshold', default=0.05, type=float, help='heatmap  threshold')
    parser.add_argument('--pafmap_threshold', default=0.01, type=float, help='pafmap threshold')
    # parser.add_argument('--dataset_type', default='', type=str, help='dataset type')
    parser.add_argument('--dataset_type', default='GenDigit', type=str, help='dataset type')

    ########################################################################################################
    # dataLoader parameter
    parser.add_argument('--num_workers', default=num_workers if not is_debug else 1, type=int, help='number of workers for data loading')

    # thread number for parallelizing cpu-bounding tensor operation
    parser.add_argument('--num_threads', default=num_threads if not is_debug else 1, type=int, help='number of workers for tensor operations')

    ########################################################################################################
    # parameter for  makeImageFolder
    # output directory parameter
    parser.add_argument('--dir_gauge_json', default=dir_gauge_json, help='directory to get gauge inform as json type')
    parser.add_argument('--dir_class_basic', default=dir_class_basic)

    # for make_train_val_from_digit_class
    # parser.add_argument('--dir_class_basic', default=dir_class_basic, help='digit base sample directory to split or for input directory')
    parser.add_argument('--train_ratio', default=0.8, help='train data rate to split the amount of image')


    # parser.add_argument('--dir_class_basic', default=dir_class_basic, help='directory to get gauge inform as json type')
    parser.add_argument('--dir_class_basic_aug', default=dir_class_basic_aug)

    parser.add_argument('--dir_digit_paf', default=dir_digit_paf)
    parser.add_argument('--dir_digit_heat', default=dir_digit_heat)

    #
    parser.add_argument('-o', '--output', default='output.json', help='output file')
    parser.add_argument('--dir_output', default=dir_output, help='output directory')

    ########################################################################################################
    # input parameter for  gen_digit
    parser.add_argument('--aug_level', default=aug_level, help='augmentation level ')
    parser.add_argument('--dir_digit_anno', default=dir_digit_anno, help='digit annotation directory')
    parser.add_argument('--dir_digit_back', default=dir_digit_back, help='digit background directory')
    parser.add_argument('--dir_digit_annoimage', default=dir_digit_annoimage, help='digit annotation and background directory')


    parser.add_argument('--gen_digit_img_load_type', default='partial_load', help='load image and annotation on dram')  # whole_load, partial_load
    parser.add_argument('--fittosize', default=FITTOSIZE, help='annotation file')  # gen_anno, read_anno
    parser.add_argument('--normal', default=0.7, help='normalize to 1. for heatmap location')

    ########################################################################################################
    # parameter for  classifier.



    ########################################################################################################
    # parameter for  pytorch DistributedDataParallel.
    parser.add_argument('--world_size', default=num_threads, type=int,help='전체 프로세스 수 - 마스터가 얼마나 많은 워커들을 기다릴지 알 수 있습니다')
    parser.add_argument('--rank', default=1, type=int,help='각 프로세스의 우선순위 - 워커의 마스터 여부를 확인할 수 있습니다')
    parser.add_argument('--dist_url', default='tcp://127.0.0.1:23456', type=str,help='url used to set up distributed training')
    parser.add_argument('--dist_backend', default=dist_backend, type=str,help='distributed backend')
    parser.add_argument('--seed', default=41, type=int,help='seed for initializing training. ')
    parser.add_argument('--gpu', default=None, type=int,help='GPU id to use.')
    parser.add_argument('--multiprocessing_distributed', action='store_true',default=True,
                        help='Use multi-processing distributed training to launch '
                             'N processes per node, which has N GPUs. This is the '
                             'fastest way to use PyTorch for either single node or '
                             'multi node data parallel training')

    ########################################################################################################
    # parameter for  pickle generation
    pickle_list_cv_heat_paf_path_augname_label_level1 = os.path.join(DIR_DATA, 'data', 'list_cv_heat_paf_path_augname_label_level1.pickle')
    pickle_list_cv_heat_paf_path_augname_label_level2 = os.path.join(DIR_DATA, 'data', 'list_cv_heat_paf_path_augname_label_level2.pickle')
    parser.add_argument('--pickle_list_cv_heat_paf_path_augname_label_level1', default=pickle_list_cv_heat_paf_path_augname_label_level1, help='annotation image pickle file name definition')
    parser.add_argument('--pickle_list_cv_heat_paf_path_augname_label_level2', default=pickle_list_cv_heat_paf_path_augname_label_level2, help='annotation image pickle file name definition')

    ########################################################################################################
    # parameter for  tqdm generation
    parser.add_argument('--verbose', default=verbose,  help='display the progress and message')
    parser.add_argument('--tqdm_desc_head', default=tqdm_desc_head, help='tqdm head message')


    args = parser.parse_args()

    # add args.device
    args.device = torch.device('cpu')
    args.pin_memory = False
    args.cuda = False
    if not args.disable_cuda and torch.cuda.is_available():
        args.device = torch.device('cuda')
        args.pin_memory = True
        args.cuda = True

    # os.makedirs(args.dir_output, exist_ok=True)
    os.makedirs(args.dir_log, exist_ok=True)

    return args


## **main_cls0.py**

In [ ]:
from pill_classifier import *
from get_cli_args import get_cli_args


job = 'resnet152'
if __name__ == '__main__':
    job = 'resnet152'
    args = get_cli_args(job=job, run_phase='test', aug_level=0, dataclass='0')

    print(f'model_path_in is {args.model_path_in}')

    end = time.time()
    if args.run_phase == 'train'  :
        args.dataset_train = Dataset_Pill(args, args.json_pill_class_list,  transform=transform_normalize, run_phase='train')
        print(f'train dataset was loaded')

    args.dataset_valid = Dataset_Pill(args, args.json_pill_class_list, transform=transform_normalize, run_phase='test' if args.run_phase == 'test' else 'valid')
    print(f'valid dataset was loaded')


    print(f'dataset loading time is {time.time() - end}')

    pill_classifier(args)
    print('job done')

ModuleNotFoundError: ignored

## **main_cls01_dir.py**

In [ ]:
from pill_classifier import *
from get_cli_args import get_cli_args
from pathlib import Path
from PIL import Image
import os

class Dataset_Dir(Dataset):
    def __init__(self, args, dir_dataset, transform=None, target_transform=None, run_phase='train'):
        self.args = args
        self.dir_dataset = dir_dataset
        self.transform = transform
        self.target_transform = target_transform

        self.list_images = [ png.name  for png in Path(dir_dataset).iterdir() if png.suffix == '.png']
        self.run_phase = run_phase

    def __len__(self):
        return len(self.list_images)

    def __getitem__(self, idx):
        image = Image.open(os.path.join(self.dir_dataset, self.list_images[idx]))
        label = 0
        path_img = self.list_images[idx]
        aug_name = ""

        if self.transform is not None:
            image = self.transform(image)

        if self.target_transform is not None:
            label = self.target_transform(label)
        if self.run_phase == 'valid' or self.run_phase == 'test':
            return image, label, path_img, aug_name
        else:
            return image, label


job = 'resnet152'
if __name__ == '__main__':
    # job = 'hrnet_w64'
    job = 'resnet152'
    args = get_cli_args(job=job, run_phase='test', aug_level=0, dataclass='01')

    print(f'model_path_in is {args.model_path_in}')

    dir_testimage = r'.\dir_testimage'

    args.dataset_valid = Dataset_Dir(args, dir_testimage, transform=transform_normalize, run_phase='test' if args.run_phase == 'test' else 'valid')
    args.batch_size = len(args.dataset_valid)
    args.verbose = False
    print(f'valid dataset was loaded')

    pill_classifier(args)

    print(args.path_img)
    print(args.list_preds)
    print('job done')

## **main_cls1.py**

In [ ]:
from pill_classifier import *
from get_cli_args import get_cli_args


job = 'resnet152'
if __name__ == '__main__':
    # job = 'hrnet_w64'
    job = 'resnet152'
    args = get_cli_args(job=job, run_phase='test', aug_level=0, dataclass='1')

    print(f'model_path_in is {args.model_path_in}')

    end = time.time()
    if args.run_phase == 'train'  :
        args.dataset_train = Dataset_Pill(args, args.json_pill_class_list,  transform=transform_normalize, run_phase='train')
        print(f'train dataset was loaded')

    args.dataset_valid = Dataset_Pill(args, args.json_pill_class_list, transform=transform_normalize, run_phase='test' if args.run_phase == 'test' else 'valid')
    print(f'valid dataset was loaded')


    print(f'dataset loading time is {time.time() - end}')

    pill_classifier(args)
    print('job done')

## **main_cls01.py**

In [ ]:
from pill_classifier import *
from get_cli_args import get_cli_args


job = 'resnet152'
if __name__ == '__main__':
    # job = 'hrnet_w64'
    job = 'resnet152'
    args = get_cli_args(job=job, run_phase='test', aug_level=0, dataclass='01')

    print(f'model_path_in is {args.model_path_in}')

    end = time.time()
    if args.run_phase == 'train'  :
        args.dataset_train = Dataset_Pill(args, args.json_pill_class_list,  transform=transform_normalize, run_phase='train')
        print(f'train dataset was loaded')

    args.dataset_valid = Dataset_Pill(args, args.json_pill_class_list, transform=transform_normalize, run_phase='test' if args.run_phase == 'test' else 'valid')
    print(f'valid dataset was loaded')


    print(f'dataset loading time is {time.time() - end}')

    pill_classifier(args)
    print('job done')

## **make_label_sharpness.py**

In [ ]:
import cv2

from get_cli_args import get_cli_args
from pathlib import Path
import utils
import numpy

def estimate_sharpness(image: numpy.array, threshold: int = 100):
    if image.ndim == 3:
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    blur_map = cv2.Laplacian(image, cv2.CV_64F)
    score = numpy.var(blur_map)
    return score

def make_label_sharpness(args):
    pathdir_dest = Path(args.dir_pill_class_base)
    dict_pill_prescription_type = utils.read_dict_from_json(args.json_pill_prescription_type)['pill_prescription_type']

    list_pills_pres_sharp_score = []
    list_pills_otc_sharp_score = []
    count = 0
    for pathdir_pill_class in pathdir_dest.iterdir():
        if not pathdir_pill_class.is_dir() :
            continue
        print(f'{count}, {pathdir_pill_class} is being estimated')
        count += 1
        list_pill_class_shape_score = []
        for  pathfile_pill_png in pathdir_pill_class.iterdir():
            if pathfile_pill_png.suffix == '.json':
                continue
            score = estimate_sharpness(utils.open_opencv_file(str(pathfile_pill_png)))
            list_pill_class_shape_score.append(score)

        score_min = min(list_pill_class_shape_score)
        score_max = max(list_pill_class_shape_score)
        score_mean = sum(list_pill_class_shape_score)/len(list_pill_class_shape_score)
        if dict_pill_prescription_type[pathdir_pill_class.stem] == 'PRES':
            list_pills_pres_sharp_score.append([str(pathdir_pill_class.stem), score_mean, score_min,score_max])
        elif dict_pill_prescription_type[pathdir_pill_class.stem] == 'OTC':
            list_pills_otc_sharp_score.append([str(pathdir_pill_class.stem), score_mean, score_min,score_max])

    list_pills_pres_sharp_score = sorted(list_pills_pres_sharp_score, key=lambda a: a[2], reverse=True)         # sharp min 기준으로 정렬.
    list_pills_otc_sharp_score = sorted(list_pills_otc_sharp_score, key=lambda a: a[2], reverse=True)           # sharp min 기준으로 정렬.
    list_pills_label_path_sharp_score = [ [label, pathname, score_mean, score_min, score_max] for label, ( pathname, score_mean, score_min, score_max ) in enumerate(list_pills_pres_sharp_score) if label < 600]
    list_pills_label_path_sharp_score += [[label+600, pathname, score_mean, score_min, score_max] for label, (pathname, score_mean, score_min, score_max) in enumerate(list_pills_otc_sharp_score) if label < 400]

    dict_temp = { 'pill_label_path_sharp_score':list_pills_label_path_sharp_score}

    utils.save_dict_to_json(dict_temp, args.json_pill_label_path_sharp_score)

def get_dict_label_pillid(args):
    dict_temp = utils.read_dict_from_json(args.json_pill_label_path_sharp_score)
    list_pills_label_path_sharp_score = dict_temp['pill_label_path_sharp_score']

    dict_label_pillid = {}
    for idx , (label, pillid, score_mean, score_min, score_max) in enumerate(list_pills_label_path_sharp_score) :
        if idx >= args.num_classes:
            break
        # print(f'{label},{pillid},{score_mean},{score_min},{score_max}')
        dict_label_pillid.update({label: pillid})

    return dict_label_pillid


def get_dict_pillid_label(args):
    dict_temp = utils.read_dict_from_json(args.json_pill_label_path_sharp_score)
    list_pills_label_path_sharp_score = dict_temp['pill_label_path_sharp_score']

    dict_label_pillid = {}
    for idx, (label, pillid, score_mean, score_min, score_max) in enumerate(list_pills_label_path_sharp_score):
        if idx >= args.num_classes:
            break
        dict_label_pillid.update({pillid:label})

    return dict_label_pillid




if __name__ == '__main__':

    # job = 'hrnet_w64'
    job = 'resnet152'
    args = get_cli_args(job=job, run_phase='train', aug_level=1, dataclass='0')
    args.logger = utils.create_logging(args.file_log)
    make_label_sharpness(args)
    # get_dict_pillid_label(args)
    # dict_label_pillid = get_dict_label_pillid(args)
    print('done')


##**make_pill_class_list.py**

In [ ]:
import shutil

from get_cli_args import get_cli_args
import utils
from pathlib import Path
import random
import os

def get_pill_info_from_pillfile(path_png):
    pill_basename, pill_status, pill_back, pill_front, pill_light, pill_lati, pill_longi, pill_dist = path_png.stem.split('_')
    pill_status, pill_back, pill_front, pill_light, pill_lati, pill_longi, pill_dist = int(pill_status), int(pill_back), int(pill_front), int(pill_light), int(pill_lati), int(pill_longi), int(pill_dist)
    return pill_basename, pill_status, pill_back, pill_front, pill_light, pill_lati, pill_longi, pill_dist

def get_pillid_from_pillfile(file_png):
    try :
        pill_basename, pill_status, pill_back, pill_front, pill_light, pill_lati, pill_longi, pill_dist = Path(file_png).stem.split('_')
    except:
        # 상위 dir의  name이  pill id가 된다.
        path_file = Path(file_png)
        pill_basename = path_file.parts[-2]
    return pill_basename

def make_pill_class_list(args):
    '''
    json_pill_sharpness 파일으로 부터, dataset을 만든다.
    class 1 :  카메라 위도, 90, 75
    class 2 : 카메라 위도 70, 65
    :param args:
    :return:
    train set : 90%
    valid set : 10%
    test set : 10%
    '''

    pill_class0 = []
    pill_class1 = []
    dict_temp = utils.read_dict_from_json(args.json_pill_label_path_sharp_score)
    list_pills_label_pillid_sharp_score = dict_temp['pill_label_path_sharp_score']
    for label, pillid, score_mean, score_min, score_max in list_pills_label_pillid_sharp_score:
        if label >= args.num_classes:
            continue
        print(f'reading sharp score.  label:{label}')
        pillid = Path(os.path.join(args.dir_pill_class_base, pillid))
        for file_png in pillid.iterdir():
            if file_png.suffix != '.png':
                continue

            pill_basename, pill_status, pill_back, pill_front, pill_light, pill_lati, pill_longi, pill_dist = get_pill_info_from_pillfile(file_png)

            if pill_lati in args.pill_dataset_class0:
                pill_class0.append(str(file_png))
            elif pill_lati in args.pill_dataset_class1:
                pill_class1.append(str(file_png))
            else:
                pass

    list_index_class0 = list(range(len(pill_class0)))
    len_train = int(round(args.pill_dataset_train_rate * len(list_index_class0)))
    list_index_class0_train = random.sample(list_index_class0, len_train)
    list_index_class0_valid = list(set(list_index_class0) - set(list_index_class0_train))
    list_index_class0_test = random.sample(list_index_class0_valid, int(round(len(list_index_class0_valid) * (args.pill_dataset_test_rate / (args.pill_dataset_test_rate + args.pill_dataset_valid_rate)))))
    list_index_class0_valid = list(set(list_index_class0_valid) - set(list_index_class0_test))

    list_index_class1 = list(range(len(pill_class1)))
    len_train = int(round(args.pill_dataset_train_rate * len(list_index_class1)))
    list_index_class1_train = random.sample(list_index_class1, len_train)
    list_index_class1_valid = list(set(list_index_class1) - set(list_index_class1_train))
    list_index_class1_test = random.sample(list_index_class1_valid, int(round(len(list_index_class1_valid) * (args.pill_dataset_test_rate / (args.pill_dataset_test_rate + args.pill_dataset_valid_rate)))))
    list_index_class1_valid = list(set(list_index_class1_valid) - set(list_index_class1_test))

    list_pngfile_class0_train = [pill_class0[index] for index in list_index_class0_train]
    list_pngfile_class0_valid = [pill_class0[index] for index in list_index_class0_valid]
    list_pngfile_class0_test = [pill_class0[index] for index in list_index_class0_test]

    list_pngfile_class1_train = [pill_class1[index] for index in list_index_class1_train]
    list_pngfile_class1_valid = [pill_class1[index] for index in list_index_class1_valid]
    list_pngfile_class1_test = [pill_class1[index] for index in list_index_class1_test]

    print(f'pngfile_class0_train:{len(list_pngfile_class0_train)}')
    print(f'pngfile_class0_valid:{len(list_pngfile_class0_valid)}')
    print(f'pngfile_class0_test:{len(list_pngfile_class0_test)}')

    print(f'pngfile_class1_train:{len(list_pngfile_class1_train)}')
    print(f'pngfile_class1_valid:{len(list_pngfile_class1_valid)}')
    print(f'pngfile_class1_test:{len(list_pngfile_class1_test)}')

    dict_temp = {'pngfile_class0_train': list_pngfile_class0_train, 'pngfile_class0_valid': list_pngfile_class0_valid, 'pngfile_class0_test': list_pngfile_class0_test, 'pngfile_class1_train': list_pngfile_class1_train, 'pngfile_class1_valid': list_pngfile_class1_valid, 'pngfile_class1_test': list_pngfile_class1_test, }

    utils.save_dict_to_json(dict_temp, args.json_pill_class_list)

def rename_non_candidate_to_s_id(args):
    # 후보가 아닌 알약 directory을  다른 이름(K head)으로 변경한다.
    dict_temp = utils.read_dict_from_json(args.json_pill_label_path_sharp_score)
    list_pills_label_pillid_sharp_score = dict_temp['pill_label_path_sharp_score']
    list_candidate_ids = [ pillid for label, pillid, score_mean, score_min, score_max in list_pills_label_pillid_sharp_score ]

    path_pill_base = Path(args.dir_pill_class_base )
    list_pill_all_id = []
    for pill_dir in path_pill_base.iterdir():
        if not pill_dir.is_dir() :
            continue
        list_pill_all_id.append(pill_dir.stem)

    list_non_candidate_pillid = list( set(list_pill_all_id) - set(list_candidate_ids))

    for pillid in list_non_candidate_pillid :
        new_id = pillid.replace('K', 'S')
        path_old = os.path.join(args.dir_pill_class_base, pillid)
        path_new = os.path.join(args.dir_pill_class_base, new_id)
        shutil.move(path_old, path_new)


if __name__ == '__main__':

    # job = 'hrnet_w64'
    job = 'resnet152'
    args = get_cli_args(job=job, run_phase='train', aug_level=1, dataclass='0')
    args.logger = utils.create_logging(args.file_log)
    # make_pill_class_list(args)
    rename_non_candidate_to_s_id(args)
    print('job done')


##**pill_classifier_hvd.py**

In [ ]:
from get_cli_args import get_cli_args
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
import torch.optim as optim
import torch.backends.cudnn as cudnn
from torch.optim.lr_scheduler import ReduceLROnPlateau

from hrnet import get_hrnet
from utils import model_load, transform_normalize, get_optimizer
import time
from tqdm import tqdm
from torch.utils.tensorboard import SummaryWriter
import torch.multiprocessing as mp
from pill_classifier import get_pill_model, Dataset_Pill, run_model
import horovod.torch as hvd


def adjust_learning_rate_hvd(args, optimizer, len_loader, epoch, batch_idx):
    if epoch < args.warmup_epochs:
        epoch += float(batch_idx + 1) / len_loader
        lr_adj = 1. / hvd.size() * (epoch * (hvd.size() - 1) / args.warmup_epochs + 1)
    elif epoch < 30:
        lr_adj = 1.
    elif epoch < 60:
        lr_adj = 1e-1
    elif epoch < 80:
        lr_adj = 1e-2
    else:
        lr_adj = 1e-3

    lr = args.base_lr * hvd.size() * args.batches_per_allreduce * lr_adj
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

    return lr

def pill_classifier_hvd(args):
    # pytorch의 DataLoader 함수를 사용할 때 num_workers>1인 경우를  지원해서, fork가 지원되게.
    torch.multiprocessing.freeze_support()
    print(f'model path is {args.model_path_in}')

    args.allreduce_batch_size = args.batch_size * args.batches_per_allreduce

    hvd.init()
    torch.manual_seed(args.seed)

    if args.cuda:
        # Horovod: pin GPU to local rank.
        torch.cuda.set_device(hvd.local_rank())
        torch.cuda.manual_seed(args.seed)
        args.gpu = hvd.local_rank()

    cudnn.benchmark = True  # 내장된 cudnn 자동 튜너를 활성화하여, 하드웨어에 맞게 사용할 최상의 알고리즘(텐서 크기나 conv 연산에 맞게?)을 찾는다.
                            # 입력 이미지 크기가 자주 변하지 않는다면, 초기 시간이 소요되지만 일반적으로 더 빠른 런타임의 효과를 볼 수 있다
    torch.backends.cudnn.deterministic = False      # 만일 True이면, cudnn에 맞추는 알고리즘이 없으면, Raise Error가 된다.
    torch.backends.cudnn.enabled = True

    verbose = args.verbose if hvd.rank() == 0 else False
    args.rank = hvd.rank()

    # Horovod: write TensorBoard logs on first worker.
    log_writer = SummaryWriter(args.dir_log) if hvd.rank() == 0 else None

    # Horovod: limit # of CPU threads to be used per worker.
    torch.set_num_threads(args.num_threads)

    kwargs = {'num_workers': args.num_workers, 'pin_memory': True} if args.cuda else {}

    # When supported, use 'forkserver' to spawn dataloader workers instead of 'fork' to prevent
    # issues with Infiniband implementations that are not fork-safe
    if (kwargs.get('num_workers', 0) > 0 and hasattr(mp, '_supports_context') and
        mp._supports_context and 'forkserver' in mp.get_all_start_methods()):
        kwargs['multiprocessing_context'] = 'forkserver'

    #############################################################################
    print("Loading dataset...")

    if args.run_phase == 'train':
        dataset_train = Dataset_Pill(args, args.json_pill_class_list, transform=transform_normalize, run_phase='train')
        sampler_train = torch.utils.data.distributed.DistributedSampler(dataset_train, num_replicas=hvd.size(), rank=hvd.rank())
        dataloader_train = DataLoader(dataset_train, batch_size=args.batch_size, sampler=sampler_train, **kwargs)

    dataset_valid = Dataset_Pill(args, args.json_pill_class_list, transform=transform_normalize, run_phase='valid')
    sampler_valid = torch.utils.data.distributed.DistributedSampler(dataset_valid, num_replicas=hvd.size(), rank=hvd.rank())
    dataloader_valid = DataLoader(dataset_valid, batch_size=args.batch_size, sampler=sampler_valid, **kwargs)


    model = get_pill_model(args)

    # define loss function (criterion) and optimizer
    criterion = torch.nn.CrossEntropyLoss().cuda()
    optimizer = get_optimizer(args, model)
    compression = hvd.Compression.fp16 if args.fp16_allreduce else hvd.Compression.none

    optimizer = hvd.DistributedOptimizer(
        optimizer, named_parameters=model.named_parameters(),
        compression=compression,
        backward_passes_per_step=args.batches_per_allreduce,
        op=hvd.Adasum if args.use_adasum else hvd.Average,
        gradient_predivide_factor=args.gradient_predivide_factor)

    epoch_begin, dict_checkpoint, success = model_load(args, model, optimizer)
    epoch_begin = hvd.broadcast(torch.tensor(epoch_begin), root_rank=0, name='epoch_begin').item()

    # Horovod: broadcast parameters & optimizer state.
    hvd.broadcast_parameters(model.state_dict(), root_rank=0)
    hvd.broadcast_optimizer_state(optimizer, root_rank=0)           # optimizer.state_dict()가 아니다. optimizer가 들어간다.


    run_model(args, model, dataloader_train, dataloader_valid, sampler_train, sampler_valid, criterion,optimizer, epoch_begin, log_writer, verbose  )




if __name__ == '__main__':
    # job = 'hrnet_w64'
    job = 'resnet152'
    args = get_cli_args(job=job, run_phase='train', aug_level=0, dataclass='0')
    pill_classifier_hvd(args)

    print('job done')

##**pill_classifier.py**

In [ ]:
from get_cli_args import get_cli_args
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
import torch.optim as optim
import torch.backends.cudnn as cudnn
from torch.optim.lr_scheduler import ReduceLROnPlateau

from gen_pill import Gen_Digit
from hrnet import get_hrnet
from utils import model_load, model_save, accuracy, get_optimizer, transform_normalize, AverageMeter, adjust_learning_rate
import time
from tqdm import tqdm
from torch.utils.tensorboard import SummaryWriter
import os
import datetime


class Dataset_Pill(Dataset):
    def __init__(self, args, dir_dataset, transform=None, target_transform=None, run_phase='train'):
        self.args = args
        self.gen_digit = Gen_Digit(args, dir_dataset, run_phase)
        self.transform = transform
        self.target_transform = target_transform
        self.run_phase = run_phase

    def __len__(self):
        return self.gen_digit.len_total

    def __getitem__(self, idx):
        image, label, path_img, aug_name = self.gen_digit.generate_digits_by_index(self.args, idx)

        if self.transform is not None:
            image = self.transform(image)

        if self.target_transform is not None:
            label = self.target_transform(label)
        if self.run_phase == 'valid' or self.run_phase == 'test':
            return image, label, path_img, aug_name
        else:
            return image, label

def get_pill_model(args):

    if args.cnn_name == 'resnet152' :
        model = models.resnet152(num_classes=args.num_classes)
    elif args.cnn_name == 'hrnet_w64' :
        model = get_hrnet()
        model.classifier = nn.Linear(in_features=2048, out_features=args.num_classes, bias=True)
    else:
        raise Exception('No Found CNN Name')

    if args.cuda == True:
        if args.gpu is not None:
            model.cuda(args.gpu)
        else:
            model.cuda()
    else:
        model.cpu()

    return model



def train(args, dataloader, sampler, model, criterion, optimizer, epoch, log_writer=None, verbose=True):
    model.train()
    if sampler != None:
        sampler.set_epoch(epoch)

    metric_train_loss = AverageMeter()
    ametric_data_time = AverageMeter()

    top1 = AverageMeter()
    top5 = AverageMeter()


    end = time.time()
    lr = optimizer.param_groups[0]['lr']
    with tqdm(total=len(dataloader), desc=args.tqdm_desc_head + 'Train Epoch  #{}'.format(epoch), disable=not verbose) as t:
        for batch_idx, (img, target) in enumerate(dataloader):
            if args.cuda:
                img = img.cuda()
                target = target.cuda()

            optimizer.zero_grad()
            output = model(img)
            loss = criterion(output, target)

            loss.backward()
            optimizer.step()

            prec1, prec5 = accuracy(output, target, (1, 5))
            count_try = img.cpu().shape[0]
            top1.update(prec1[0].detach().cpu().item(), count_try)
            top5.update(prec5[0].detach().cpu().item(), count_try)

            metric_train_loss.update(loss.detach().cpu().item(), count_try)
            t.set_postfix({'loss': metric_train_loss.avg, 'lr':lr,  'top1':top1.avg, 'top5':top5.avg })
            t.update(1)

    ametric_data_time.update(time.time() - end)

    if log_writer:
        log_writer.add_scalar('train/loss', metric_train_loss.avg, epoch)

    try:
        print_string = 'Epoch: [{0}][{1}/{2}]\t'.format(epoch, batch_idx, len(dataloader))
        print_string += 'Data time {data_time.val:.3f} ({data_time.avg:.3f} Now:{Now})\t'.format(data_time=ametric_data_time, Now=datetime.datetime.now())
        print_string += 'Loss {loss.val:.4f} ({loss.avg:.4f})'.format(loss=metric_train_loss)
        print_string += 'Accuracy top1:{top1.avg:.4f}, top5:{top5.avg:.4f}'.format(top1=top1, top5=top5)
        print(print_string)
    except:
        pass

    return metric_train_loss.avg


def valid(args, dataloader, sampler,  model, criterion, epoch, log_writer=None, verbose=True):
    metric_train_loss = AverageMeter()
    ametric_data_time = AverageMeter()

    top1 = AverageMeter()
    top5 = AverageMeter()
    if sampler != None:
        sampler.set_epoch(epoch)
    model.eval()

    end = time.time()
    args.list_preds = []
    args.list_target = []
    args.count_correct = 0
    with torch.no_grad():
        with tqdm(total=len(dataloader), desc=args.tqdm_desc_head + '{} Epoch  #{}'.format( args.run_phase, epoch), disable=not verbose) as t:
            for i, (img, target, path_img, aug_name ) in enumerate(dataloader):
                # measure data loading time

                if args.cuda:
                    img = img.cuda()
                    target = target.cuda()

                output = model(img)
                loss = criterion(output, target)

                prec1, prec5 = accuracy(output, target, (1, 5))

                if ( prec1[0].detach().cpu().item() != 100.):
                    if args.run_phase == 'valid': print(f'<------- class valid fail file: {path_img[0]}, aug_name:{aug_name}')

                preds = output.data.max(dim=1, keepdim=True)[1]
                count_correct = preds.eq(target.data.view_as(preds)).cpu().sum()
                list_preds = preds.view(-1).tolist()

                args.list_preds = list_preds
                args.list_target = target.detach().cpu().tolist()
                args.count_correct = count_correct.item()
                count_try = img.cpu().shape[0]
                top1.update(prec1[0].detach().cpu().item(), count_try)
                top5.update(prec5[0].detach().cpu().item(), count_try)

                metric_train_loss.update(loss.detach().cpu().item(), count_try)

                t.set_postfix({'loss': metric_train_loss.avg, 'top1': top1.avg, 'top5': top5.avg})
                t.update(1)

    ametric_data_time.update(time.time() - end)

    if log_writer:
        log_writer.add_scalar('validation/loss', metric_train_loss.avg, epoch)

    try:
        print_string = 'Epoch: [{0}][{1}/{2}]\t'.format(epoch, i, len(dataloader))
        print_string += 'Data time {data_time.val:.3f} ({data_time.avg:.3f} Now:{Now})\t'.format(data_time=ametric_data_time, Now=datetime.datetime.now())
        print_string += 'Loss {loss.val:.4f} ({loss.avg:.4f})'.format(loss=metric_train_loss)
        print_string += 'Accuracy top1:{top1.avg:.4f}, top5:{top5.avg:.4f}'.format(top1=top1, top5=top5)
        print(print_string)
    except:
        pass

    return metric_train_loss.avg

def run_model(args, model, dataloader_train, dataloader_valid, sampler_train, sampler_valid, criterion,optimizer, epoch_begin, log_writer, verbose=True  ):
    if args.run_phase == 'valid' or args.run_phase == 'test':
        print(time.asctime())
        valid(args, dataloader_valid, sampler_valid,  model, criterion, 0, log_writer)
        print(time.asctime())
        return

    lr_scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.8, patience=2, verbose=True, threshold=0.0001, threshold_mode='rel', cooldown=3, min_lr=0, eps=1e-08)


    best_perf = 1000
    for epoch in range(epoch_begin, args.epochs):   # ( 0:100)
        # adjust_learning_rate(args, optimizer, epoch)
        # train for one epoch
        perf_indicator = train(args, dataloader_train, sampler_train, model, criterion, optimizer, epoch, log_writer, verbose)
        if epoch > 10:
            perf_indicator = valid(args, dataloader_valid,sampler_valid, model, criterion, epoch, log_writer,verbose)
            if (args.gpu == 0):
                print(f'perf_indicator:{perf_indicator} ,  best_perf:{best_perf}')
            if perf_indicator < best_perf:
                model_save(args.model_path, epoch, model, optimizer, args.rank)
                best_perf = perf_indicator
        else:
            model_save(args.model_path, epoch, model, optimizer, args.rank)
            best_perf = perf_indicator

        lr_scheduler.step(perf_indicator)


model = None
criterion = None
optimizer = None
epoch_begin = 0
log_writer = None

def pill_classifier(args):
    global model, criterion, optimizer, epoch_begin, log_writer
    if args.dataset_valid != None:
        dataloader_valid = DataLoader(args.dataset_valid, batch_size=args.batch_size, shuffle= False, num_workers=args.num_workers)
    else:
        dataloader_valid = None

    if args.run_phase == 'train' and args.dataset_train != None:
        dataloader_train = DataLoader(args.dataset_train, batch_size=args.batch_size, shuffle=True, num_workers=args.num_workers)
    else:
        dataloader_train = None

    if model == None :
        log_writer = SummaryWriter(args.dir_log)
        if args.cuda == False or torch.cuda.device_count() == 0  :
            args.gpu = None
        else:
            args.gpu = 0

        args.rank = args.gpu

        cudnn.benchmark = True
        torch.backends.cudnn.deterministic = False
        torch.backends.cudnn.enabled = True

        model = get_pill_model(args)

        # define loss function (criterion) and optimizer
        criterion = torch.nn.CrossEntropyLoss()
        if args.cuda:
            criterion = criterion.cuda()
        optimizer = get_optimizer(args,model)
        epoch_begin, dict_checkpoint, success = model_load(args, model, optimizer)

    run_model(args, model, dataloader_train, dataloader_valid, None, None,  criterion,optimizer, epoch_begin, log_writer )


if __name__ == '__main__':
    # job = 'hrnet_w64'
    job = 'resnet152'
    args = get_cli_args(job=job, run_phase='test', aug_level=0, dataclass='0')

    print(f'model_path_in is {args.model_path_in}')

    end = time.time()
    if args.run_phase == 'train'  :
        args.dataset_train = Dataset_Pill(args, args.json_pill_class_list,  transform=transform_normalize, run_phase='train')
        print(f'train dataset was loaded')

    args.dataset_valid = Dataset_Pill(args, args.json_pill_class_list, transform=transform_normalize, run_phase='test' if args.run_phase == 'test' else 'valid')
    print(f'valid dataset was loaded')


    print(f'dataset loading time is {time.time() - end}')

    pill_classifier(args)
    print('job done')

##**utils.py**

In [ ]:
import codecs, json
import numpy as np
import cv2
import torch
import os
from PIL import Image
from pathlib import Path
from collections import OrderedDict
from torch.utils.data import Dataset
from torchvision import transforms
import torch.optim as optim
from glob import glob
import matplotlib.pyplot as plt
import matplotlib
import logging


def inverse_vgg_preprocess(image):
    means = [0.485, 0.456, 0.406]
    stds = [0.229, 0.224, 0.225]
    image = image.copy().transpose((1, 2, 0))  # 원본 손상 방지.

    for i in range(3):
        image[:, :, i] = image[:, :, i] * stds[i]
        image[:, :, i] = image[:, :, i] + means[i]
    image = image[:, :, ::-1]
    image = image * 255

    image[image > 255.] = 255.
    image[image < 0.] = 0.
    image = image.astype(np.uint8)

    return image

##########################################################################
def save_dict_to_json(dict_save, filejson, mode='w'):
    with codecs.open(filejson, mode, encoding='utf-8') as f:
        json.dump(dict_save, f, ensure_ascii=False, indent=4)


def read_dict_from_json(filejson):
    if not os.path.isfile(filejson) :
        return None
    with codecs.open(filejson, 'r', encoding='utf-8') as f:
        obj = json.load(f)
        return obj

def open_opencv_file(filename):
    img_array = np.fromfile(filename, np.uint8)
    image = cv2.imdecode(img_array, cv2.IMREAD_COLOR)
    return image

def save_opencv_file(image, filename):
    result, encoded_img = cv2.imencode('.jpg', image)
    if result:
        with open(filename, mode='w+b') as f:
            encoded_img.tofile(f)
            return True
    else:
        return False

def show_cvimage(image):
    cv2.imshow('a', image)
    cv2.waitKey()                   # wait을 해야, 이미지가 나온다. PIL은  im.show()으로 바로 나온다.
    cv2.destroyAllWindows()


def show_tensor3(inp, cmap=None):
    """Imshow for Tensor."""
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt.imshow(inp, cmap)
    plt.show()
    plt.close()


def save_tensor3(inp, filename):
    """Imshow for Tensor."""
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    matplotlib.image.imsave(filename, inp)



def model_save(model_path, epoch, model, optimizer, rank=0) :
    if rank == 0 :
        torch.save({
            'epoch': epoch,
            'model': model.state_dict(),
            'optimizer': optimizer.state_dict() if optimizer != None else 0 ,
        }, model_path)
        print(f'model was saved to {model_path}')

def model_load(args, model, optimizer, rank=0) :
    epoch_begin = 0
    state_model = None
    state_optimizer = None
    if not os.path.isfile(args.model_path_in):
        print(f"-------------------------------------------------->>>>>>>>>>>>> model_path doesn't exist:{args.model_path_in}")
        return  epoch_begin, None, False      # None check point to indicate a fail

    if args.verbose == True: print(f"model_path will be loaded from:{args.model_path_in}")

    dict_checkpoint = torch.load(args.model_path_in, map_location='cpu')

    epoch_begin = dict_checkpoint.get('epoch', -1 ) + 1
    if rank == 0 :
        try:
            state_model = dict_checkpoint.get('model', None )
            if state_model != None :
                if not hasattr(model, 'module') and ('module' in list(state_model.keys())[0] ) :           # model_path은  module 을 포함하고 있다고 가정한다.
                    state_model = OrderedDict([ ( k[7:], v ) if 'module' in k else (k,v ) for k,v in state_model.items()])
                    model.load_state_dict(state_model)
                    print(f'model was loaded from module state')
                elif hasattr(model, 'module') and (not 'module' in list(state_model.keys())[0] ) :
                    state_model = OrderedDict([('module.' + k, v)  for k, v in state_model.items()])
                    model.load_state_dict(state_model)
                    print(f'module.model was loaded from normal state')
                else:
                    model.load_state_dict(state_model)
                    print(f'model was loaded from state')
            else:
                print(f'No model checkpoint in file')

        except Exception as e :
            print(f'Fail to loading model: {e}')
            return epoch_begin, dict_checkpoint, False

        if optimizer != None and args.run_phase == 'train':
            try:
                state_optimizer = dict_checkpoint.get('optimizer', None )
                if state_optimizer != None:
                    optimizer.load_state_dict(state_optimizer)
                    print(f'optimizer was loaded from state')
                else:
                    print(f'No optimizer checkpoint in file')
            except Exception as e :
                print(f'Fail to loading optimizer: {e}')
                return epoch_begin, dict_checkpoint, False
    success = True if state_model != None and state_optimizer != None else False
    return epoch_begin, dict_checkpoint, success


def convert_pil_to_cv2(pil_image):
    pil_image = pil_image.convert('RGB')
    open_cv_image = np.array(pil_image)
    # Convert RGB to BGR
    open_cv_image = open_cv_image[:, :, ::-1].copy()
    return open_cv_image

def convert_cv2_to_pil(cv_image):
    pil_img = Image.fromarray(cv_image)
    return pil_img

def save_img_paf_heat(path, img_temp, paf_temp, heatmap_temp, args ):
    path = Path(path)

    if path.parts[-2].isdigit() :
        dir_paf = os.path.join(args.dir_review_paf_train, path.parts[-2] )
        dir_heat = os.path.join(args.dir_review_heat_train, path.parts[-2])
        dir_img = os.path.join(args.dir_review_img_train, path.parts[-2])
    else:
        dir_paf = args.dir_output
        dir_heat = args.dir_output
        dir_img = args.dir_output

    os.makedirs(dir_paf, exist_ok=True)
    os.makedirs(dir_heat, exist_ok=True)
    os.makedirs(dir_img, exist_ok=True)

    file_base = path.name.split('.')[0]
    filename_img = os.path.join(dir_img, file_base + '_base.jpg')
    filename_paf = os.path.join(dir_paf, file_base + '_paf.jpg')
    filename_heat = os.path.join(dir_heat, file_base + '_heat.jpg')

    save_opencv_file(inverse_vgg_preprocess(img_temp), filename_img)
    save_opencv_file(inverse_vgg_preprocess(paf_temp), filename_paf)
    save_opencv_file(inverse_vgg_preprocess(heatmap_temp), filename_heat)

def open_pil_as_stack_gray_np(filename):
    np_pil = np.array(Image.open(filename).convert('L'))
    np_pil = np.dstack([np_pil,np_pil,np_pil])
    return np_pil

def open_pil_as_stack_color_np(filename):
    np_pil = np.array(Image.open(filename))
    return np_pil

def save_np_pil_file(np_image, filename):
    image_pil = Image.fromarray(np_image)
    image_pil.save(filename)

def accuracy(output, target, topk=(1,)):
    """Computes the precision@k for the specified values of k"""
    with torch.no_grad():
        maxk = max(topk)
        batch_size = target.size(0)

        _, pred = output.topk(maxk, 1, True, True)      #  maxk, dim=1, largest, sorted -> ( value tensor, index longTensor )
        pred = pred.t()
        correct = pred.eq(target.view(1, -1).expand_as(pred))

        res = []
        for k in topk:
            correct_k = correct[:k].reshape(-1).float().sum(0, keepdim=True)
            res.append(correct_k.mul_(100.0 / batch_size))
        return res


def saveimage(sub, dir_digit, basename):
    filename_jpg = os.path.join(dir_digit, basename + '.jpg')
    sub.save(filename_jpg)


def extractDigit_saveto(file_json, file_bmp, list_dir_digit=None):
    dict_bmp_info = read_dict_from_json(file_json)

    digitFractNo = int(dict_bmp_info['digitFractNo'])
    digitAllNo = int(dict_bmp_info['digitAllNo'])
    dataValue = int(dict_bmp_info['dataValue'] * 10 ** digitFractNo)
    digitRect = dict_bmp_info['digitRect']
    str_dataValue = f'{dataValue:0{digitAllNo}}'
    str_igmsGaugeDataId = dict_bmp_info['igmsGaugeDataId']

    if len(str_dataValue) != digitAllNo:
        if len(str_igmsGaugeDataId) == digitAllNo:
            str_dataValue = str_igmsGaugeDataId
        else:
            print(f'{file_json}')
            raise Exception("improper data format")


    list_digitRect = digitRect.split('|')[1:digitAllNo+1]
    list_digitRect = [aa.split(',') for aa in list_digitRect]
    list_digitRect = [[int(a), int(b), int(c), int(d)] for a, b, c, d in list_digitRect]

    img = Image.open(file_bmp)
    if img == None:
        print(f"Can't read a image file :{file_bmp}")
        return

    list_image = []
    for index in range(digitAllNo):
        x, y, width, height = list_digitRect[index]
        sub = img.crop((x, y, x + width, y + height))
        if list_dir_digit != None :
            saveimage(sub, list_dir_digit[int(str_dataValue[index])], os.path.basename(file_json).split('.')[0] + f'_{index}{str_dataValue[index]}')
        else:
            list_image.append(sub.convert('RGB'))

    if list_dir_digit == None :
        return list_image, [int(aa) for aa in str_dataValue], dict_bmp_info

def get_Image_Value_List_from_json(file_json):
    list_image, list_value, dict_json_info = extractDigit_saveto(file_json, os.path.splitext(file_json)[0] + '.bmp')
    list_cv_label_path = [(list_image[i], list_value[i], file_json) for i in range(len(list_image))]
    return list_cv_label_path

transform_normalize = transforms.Compose([
                                 transforms.ToTensor(),
                                 transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                                      std=[0.229, 0.224, 0.225])
                             ])

transform_classifier = transforms.Compose([transforms.Resize((224, 224))
                                         , transforms.ToTensor()
                                         , transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
                                      ])

transfrom_paf = transforms.Compose([transforms.Resize((368, 368))
                                         , transforms.ToTensor()
                                         , transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
                                      ])

class Dataset_valid(Dataset):
    def __init__(self, file_json, transform):
        if os.path.isfile(file_json) and file_json.split('.')[-1] == 'json':
            self.list_cv_label_path = get_Image_Value_List_from_json(file_json)
        elif os.path.isdir(file_json):
            self.list_cv_label_path = [(Image.open(aa).convert("RGB"), int(str(Path(aa).parts[-2])), aa) for aa in glob(file_json + r'/**/*.jpg')]
        self.transform = transform
        self.file_json = file_json

    def __len__(self):
        # return size of dataset
        return len(self.list_cv_label_path)

    def __getitem__(self, idx):
        image, label, path = self.list_cv_label_path[idx]
        if self.transform != None:
            image = self.transform(image)
        return image, (label, path)

def get_optimizer(args, model):
    optimizer = None
    if args.optimizer == 'sgd':
        optimizer = optim.SGD(
            filter(lambda p: p.requires_grad, model.parameters()),
            lr=args.lr,
            momentum=args.momentum,
            weight_decay=args.wd,
            nesterov=args.nesterov
        )
        if args.verbose == True: print(f'optimizer was selected as type:sgd')
    elif args.optimizer == 'adam':
        optimizer = optim.Adam( filter(lambda p: p.requires_grad, model.parameters()), lr=args.lr)
        if args.verbose == True: print(f'optimizer was selected as type:adam')
    elif args.optimizer == 'rmsprop':
        optimizer = optim.RMSprop(
            filter(lambda p: p.requires_grad, model.parameters()),
            lr=args.lr,
            momentum=args.momentum,
            weight_decay=args.wd,
        )
        if args.verbose == True: print(f'optimizer was selected as type:rmsprop')
    return optimizer

def adjust_learning_rate(args, optimizer, epoch):
    assert hasattr(args, 'lr_schedule'), "args doesn't have lr schedule"
    assert hasattr(args, 'lr_gamma'), "args doesn't have lr gamma"
    assert hasattr(args, 'lr'), "args doesn't have lr"

    if epoch in args.lr_schedule:
        args.lr *= args.lr_gamma
        for param_group in optimizer.param_groups:
            param_group['lr'] = args.lr



class AverageMeter(object):
    """  val을 계속 누적하고,  평균을 구함."""

    def __init__(self):
        self.reset()

    def reset(self):
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.sum += val * n
        self.count += n

    @property
    def avg(self):
        return self.sum / self.count

    @property
    def val(self):
        return self.sum

def create_logging(file_log):
    path_file_log = Path(file_log)
    path_dir_log = path_file_log.parent
    path_dir_log.mkdir(exist_ok=True)

    logger = logging.getLogger()
    logger.setLevel(logging.DEBUG)

    formatter_stream = logging.Formatter(u'%(message)s')
    streamingHandler = logging.StreamHandler()
    streamingHandler.setFormatter(formatter_stream)

    formatter_file = logging.Formatter(u'%(asctime)s [%(levelname)8s] %(message)s')
    file_handler = logging.FileHandler(file_log)
    file_handler.setFormatter(formatter_file)

    logger.addHandler(streamingHandler)
    logger.addHandler(file_handler)

    return logger

